In [251]:
# Import Libraries and Data
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 300)

# Import Data
df = pd.read_csv("../data/pipeline/merged_team_season.csv")

# Create Features, Including Difference and Ratio Columns
def calculate_diff_ratio_pythag(df: pd.DataFrame, features = [str], diff_ratio_pythag: str = 'difference', home_away: bool = False):
    for feature in features:
        if home_away == False:
            if diff_ratio_pythag == 'difference':
                df[f'{feature}_diff'] = df[f'team1_{feature}'] - df[f'team2_{feature}']
            elif diff_ratio_pythag == 'ratio':
                df[f'{feature}_ratio'] = df[f'team1_{feature}'] / df[f'team2_{feature}']   
            elif diff_ratio_pythag == 'pythag':
                df['team1_pythag'] = (df[f'team1_{feature}'] - (df[f'team1_{feature}']*df[f'team2_{feature}']))/ (df[f'team1_{feature}'] + df[f'team2_{feature}']-(2*df[f'team1_{feature}']*df[f'team2_{feature}']))
        else:
            if diff_ratio_pythag == 'difference':
                df[f'team1_ha_{feature}_diff'] = df[f'team1_home_{feature}'] - df[f'team1_away_{feature}']
                df[f'team2_ha_{feature}_diff'] = df[f'team2_home_{feature}'] - df[f'team2_away_{feature}'] 
                df[f'{feature}_diff'] = df[f'team1_ha_{feature}_diff'] - df[f'team2_ha_{feature}_diff'] 
            elif diff_ratio_pythag == 'ratio':
                df[f'team1_ha_{feature}_ratio'] = df[f'team1_home_{feature}'] / df[f'team1_away_{feature}']
                df[f'team2_ha_{feature}_ratio'] = df[f'team2_home_{feature}'] / df[f'team2_away_{feature}'] 
                df[f'{feature}_ratio'] = df[f'team1_ha_{feature}_diff'] / df[f'team2_ha_{feature}_diff'] 
    return df

# Flips DF2 Differences and Ratios
def flip_df2_diff_ratio_pythag(df: pd.DataFrame, feature_dict: dict):
    
    # Loop Through Difference and Ratio, Apply Appropriate Transformation
    for diff_ratio_pythag in list(feature_dict.keys()):
        for feat in feature_dict[diff_ratio_pythag]:
            if diff_ratio_pythag == 'diff':
                df[f'{feat}_diff'] = df[f'{feat}_diff'] * -1
            elif diff_ratio_pythag == 'ratio':
                df[f'{feat}_ratio'] = 1 / df[f'{feat}_ratio']
            elif diff_ratio_pythag == 'pythag':
                df[f'team1_{diff_ratio_pythag}'] = 1 - df[f'team1_{diff_ratio_pythag}']
                
    # Change Team1 Win and Game ID
    df['team1_win'] = 0
    df['game_id'] = (df['game_id'].str.split('-', expand=True)[0] + '-' + df['game_id'].str.split('-', expand=True)[2] + '-' + df['game_id'].str.split('-', expand=True)[1])
    
    return df

In [252]:
df.head(1)

,team1_id,team1_score,team2_id,team2_score,WLoc,num_ot,team1_position,team2_position,team1_seed,team2_seed,strongseed,weakseed,team1_region,team2_region,slot,team1_teamname,team2_teamname,season,host,host_lat,host_long,team1_lat,team1_long,team2_lat,team2_long,team1_coach_id,team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,team1_pt_overall_ff,team1_pt_career_school_wins,team1_pt_career_school_losses,team1_pt_career_overall_wins,team1_pt_career_overall_losses,team1_pt_team_season_wins,team1_pt_team_season_losses,team1_pt_coach_season_wins,team1_pt_coach_season_losses,team2_coach_id,team2_pt_school_ncaa,team2_pt_overall_ncaa,team2_pt_school_s16,team2_pt_overall_s16,team2_pt_school_ff,team2_pt_overall_ff,team2_pt_career_school_wins,team2_pt_career_school_losses,team2_pt_career_overall_wins,team2_pt_career_overall_losses,team2_pt_team_season_wins,team2_pt_team_season_losses,team2_pt_coach_season_wins,team2_pt_coach_season_losses,team1_ap_final,team1_ap_preseason,team1_coaches_before_final,team1_coaches_preseason,team2_ap_final,team2_ap_preseason,team2_coaches_before_final,team2_coaches_preseason,team1_fg2pct,team1_fg3pct,team1_ftpct,team1_blockpct,team1_oppfg2pct,team1_oppfg3pct,team1_oppftpct,team1_oppblockpct,team1_f3grate,team1_oppf3grate,team1_arate,team1_opparate,team1_stlrate,team1_oppstlrate,team2_fg2pct,team2_fg3pct,team2_ftpct,team2_blockpct,team2_oppfg2pct,team2_oppfg3pct,team2_oppftpct,team2_oppblockpct,team2_f3grate,team2_oppf3grate,team2_arate,team2_opparate,team2_stlrate,team2_oppstlrate,team1_tempo,team1_adjtempo,team1_oe,team1_adjoe,team1_de,team1_adjde,team2_tempo,team2_adjtempo,team2_oe,team2_adjoe,team2_de,team2_adjde,game_id,team1_team_id,team2_team_id,round,team1_team no,team1_team id,team1_dunks fg%,team1_dunks share,team1_dunks fg%d,team1_dunks d share,team1_close twos fg%,team1_close twos share,team1_close twos fg%d,team1_close twos d share,team1_farther twos fg%,team1_farther twos share,team1_farther twos fg%d,team1_farther twos d share,team1_threes fg%,team1_threes share,team1_threes fg%d,team1_threes d share,team2_team no,team2_team id,team2_dunks fg%,team2_dunks share,team2_dunks fg%d,team2_dunks d share,team2_close twos fg%,team2_close twos share,team2_close twos fg%d,team2_close twos d share,team2_farther twos fg%,team2_farther twos share,team2_farther twos fg%d,team2_farther twos d share,team2_threes fg%,team2_threes share,team2_threes fg%d,team2_threes d share
0,1314,81,1181,77,N,0,W08,X02,8,2,2,8,W,X,R5WX,north carolina,duke,2022,new_orleans,29.9667,-90.05,35.912165,-79.050969,36.00159,-78.94226,hubert-davis-1,0.0,0.0,0.0,0.0,0.0,0.0,24.0,9,24.0,9,24.0,9,24.0,9,mike-krzyzewski-1,35.0,35.0,25.0,25.0,12.0,12.0,1125.0,308,1198.0,367,28.0,6,28.0,6,NaN,19.0,NaN,20.0,9.0,9.0,10.0,9.0,50.714286,36.206897,77.21943,9.814529,48.299845,34.920635,73.696145,8.492063,37.437934,36.878049,53.618421,48.031496,0.076692,0.093829,55.844156,36.821192,72.881356,13.25811,46.897038,31.396957,67.160494,8.097785,36.579457,33.769267,56.987116,51.793722,0.094448,0.091014,70.2181,70.1745,109.416,113.035,100.974,96.9911,68.2765,67.4185,117.152,119.357,98.2346,95.6444,2022-1314-1181,1314,1181,5,904.0,135.0,86.8,8.6,88.3,4.9,61.5,35.6,59.0,32.4,34.6,26.4,35.7,31.4,35.8,38.0,33.7,36.2,927.0,50.0,89.7,13.4,86.2,7.5,66.7,38.7,55.4,36.5,39.8,25.6,35.9,29.9,36.6,35.7,32.0,33.6


In [253]:
# Choose Season Cutoff and Features

# Limit Years
df = df[df['season'] >= 2008]

# Calculate Differences and Ratios Between Features
features = {'diff': ['kpi_resume','sor_resume', 'avg_resume', 'bpi_quality', 'kp_quality', 'sag_quality', 'avg_quality', 'resume', 'wab', 'elo', 'power'],
            'ratio': ['adjoe', 'adjde', 'adjtempo'],
            'pythag': ['exp_win']}

In [254]:
# Calculate Pythagorean W/L Based On Adjusted Offensive Efficiency
df['team1_exp_win'] = (df['team1_adjoe']**11.5)/ ((df['team1_adjde']**11.5)+(df['team1_adjoe']**11.5))
df['team2_exp_win'] = (df['team2_adjoe']**11.5)/ ((df['team2_adjde']**11.5)+(df['team2_adjoe']**11.5))

# Calculate Team Differences, Ratios, and Pythag For AdjOE and AdjDE
df = calculate_diff_ratio_pythag(df, features['diff'], diff_ratio_pythag = 'difference')
df = calculate_diff_ratio_pythag(df, features['ratio'], diff_ratio_pythag = 'ratio')
df = calculate_diff_ratio_pythag(df, features['pythag'], diff_ratio_pythag = 'pythag')

# Establish Training Columns
standard_training_columns = ['game_id','season','team1_win','team1_score','team2_score']
feature_columns = [col for col in df.columns if 'diff' in col or 'ratio' in col or 'pythag' in col]
training_columns = standard_training_columns + feature_columns

#Create a team 1 win column to determine which rows actually have them winning
df['team1_win'] = (df['team1_score'] > df['team2_score']).astype(int)

# Randomly Select Games To Place in Df 1 and Rest in Df 2
df_1 = df.sample(frac=.5, random_state=15)[training_columns].reset_index(drop=True)
df_2 = df[~df.index.isin(df_1.index)][training_columns].reset_index(drop=True)

# In Second Df, Flip The Attributes
df_2 = flip_df2_diff_ratio_pythag(df_2, features)

#Combine DataFrames
df = pd.concat([df_2, df_1]).reset_index(drop=True)
df_2019 = df[df['season'] != 2019]
df = df[df['season'] != 2019]

# Print Checks On Split
for feature in feature_columns:
    print("Team 1 (Winning) and Team 2 (Losing):\n", df.groupby('team1_win')[feature].mean(),"\n")
    print(df.groupby('team1_win')[feature].mean())
    
# Export
df.to_csv("../data/pipeline/test_train.csv", index = False)
df_2019.to_csv("../data/pipeline/2019_validation.csv", index = False)

KeyError: 'team1_kpi_resume'

In [237]:
df.shape

(918, 21)